# API para el uso de Pytrends 


estructura general 

In [1]:
from fastapi import FastAPI
from pytrends.request import TrendReq
import pandas as pd

app = FastAPI()

# Define a list of country codes 
COUNTRY_LIST = ['AT', 'BE', 'CH', 'CY', 'CZ', 'DE', 'DK', 'EE', 'GR', 'ES',
                'FI', 'FR', 'HU', 'IE', 'IS', 'IT', 'LU', 'ME',
                'MT', 'NL', 'NO', 'PL', 'PT', 'SE', 'SI', 'GB',
                'LT', 'LV']

def fetch_google_trends_data(category, countries=COUNTRY_LIST, start_date='2014-01-01', end_date='2024-03-31'):
    pytrends = TrendReq(hl='es', tz=360)
    trends_data = {}
    for country in countries:
        try:
            pytrends.build_payload(kw_list=[category], timeframe=f'{start_date} {end_date}', geo=country)
            data = pytrends.interest_over_time()
            if data.empty:
                raise ValueError(f"No data available for the category and country '{country}'.")
            weekly_data = data.resample('W').mean()
            trends_data[country] = weekly_data.to_dict()
        except Exception as e:
            print(f"Error fetching Google Trends data for the country '{country}': {e}")
    return trends_data

@app.get("/get-google-trends/{category}")
def get_google_trends(category: str, start_date: str = '2014-01-01', end_date: str = '2024-03-31'):
    # Retrieve Google Trends data for the specified category or topic depends 
    trends_data = fetch_google_trends_data(category, COUNTRY_LIST, start_date, end_date)
    return trends_data
# also here we might need to convert it from a dict to csv 
# Convert 'datos' dictionary to a DataFrame
df = pd.DataFrame.from_dict()

# Convert the DataFrame to a CSV file
df.to_csv('datos.csv', index=True)
df


def clean_dataframe(df: pd.DataFrame):
    cleaned_df = df.dropna().drop_duplicates()
    return cleaned_df

#todavia no lo hemos comprobado "/clean-google-trends-data/"


@app.get("/clean-google-trends-data/")
def clean_google_trends_data(category: str):
    # Fetch Google Trends data
    trends_data = fetch_google_trends_data(category)
    # Convert to DataFrame
    df = pd.DataFrame.from_dict(trends_data)
    # Clean DataFrame
    cleaned_df = clean_dataframe(df)
    # Save cleaned data to CSV
    cleaned_df.to_csv(f"{category}_cleaned_data.csv", index=True)
    return {"message": f"Data cleaned and saved successfully"}



une ejemplo de uso 

In [ ]:
from pytrends.request import TrendReq
from fastapi import FastAPI
import pandas as pd
import time

app = FastAPI()

country_list = ['AT', 'BE', 'CH', 'CY','CZ' 'DE', 'DK', 'EE', 'GR', 'ES',
       'FI', 'FR', 'HU', 'IE', 'IS', 'IT', 'LU', 'ME',
       'MT', 'NL', 'NO', 'PL', 'PT', 'SE', 'SI','UK','GB',
        'LT', 'LV']

def obtener_datos_google_trends_semanal(palabra_clave, paises=country_list, fecha_inicio='2014-01-01', fecha_fin='2024-03-31'):
    try:
        pytrends = TrendReq(hl='es', tz=360)
        trends_data = {}
        for pais in paises:
            try:
                pytrends.build_payload(kw_list=[palabra_clave], timeframe=f'{fecha_inicio} {fecha_fin}', geo=pais)
                data = pytrends.interest_over_time()
                if data.empty:
                    raise ValueError(f"No hay datos disponibles para la palabra clave y país '{pais}'.")
                data_semanal = data.resample('W').mean()
                trends_data[pais] = data_semanal.to_dict()
            except Exception as e:
                print(f"Error al obtener datos de Google Trends para el país '{pais}': {e}")
                # Break off the loop if an error occurs for a country
                break
        return trends_data
    except Exception as e:
        print(f"Error al obtener datos de Google Trends: {e}")
        time.sleep(60)
        return obtener_datos_google_trends_semanal(palabra_clave, paises, fecha_inicio, fecha_fin)

@app.get("/google-trends/{topic}")
def google_trends_handler(topic: str, fecha_inicio: str = '2014-01-01', fecha_fin: str = '2024-03-31'):
    trends_data = obtener_datos_google_trends_semanal(topic, country_list, fecha_inicio, fecha_fin)
    return trends_data


palabra_clave = "Unemployment benefits"
datos = obtener_datos_google_trends_semanal(palabra_clave)
datos
